# HW 7
# smolkina julia

Докинг низкомолекулярных лигандов в структуру белка

Традиционные ссылки на полезные ресурсы:

Видеурок по работе с Autodock Vina [[http://vina.scripps.edu/tutorial.html
Статья по ODDT https://jcheminf.springeropen.com/articles/10.1186/s13321-015-0078-2
Вся работа по расчётам будет проходить на через ipython notebook.

Введение

Цель данного занятия ознакомится с возможностями докинга низкомолекулярного лиганда в структуру белка.

В этом занятии мы будем пользоваться пакетом Autodock Vina и oddt. Это программное обеспечение распространяется бесплатно для академических пользователей.

Объект

Вы будете работать с белком лизоцимом структуру которого вы построили на основе гомологичного моделирования на прошлом практикуме.

Надо определить место докинга, удалить лигнад, запустить докинг и провести анализ

In [2]:
!pip install oddt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for oddt: filename=oddt-0.7-py2.py3-none-any.whl size=2409540 sha256=1adafeacf4812adf5e73124e0f751e871536dfb563fdb5e433eca924b41ee891
  Stored in directory: /Users/mac-julia/Library/Caches/pip/wheels/9d/6c/91/07746be9751a33b4e88e4c90ed832a56f0515d0713fdcaedba
Successfully built oddt


In [9]:
!git clone https://github.com/deGrootLab/pmx.git

Cloning into 'pmx'...
remote: Enumerating objects: 17532, done.
remote: Counting objects: 100% (1205/1205), done.
remote: Compressing objects: 100% (423/423), done.
remote: Total 17532 (delta 691), reused 1140 (delta 636), pack-reused 16327
Receiving objects: 100% (17532/17532), 217.37 MiB | 2.37 MiB/s, done.
Resolving deltas: 100% (10551/10551), done.
Updating files: 100% (7323/7323), done.


In [17]:
!conda install jupyter schrodinger::pymol

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found to be incompatible with each other:

Output in format: Requested package -> Available versions

Package pyqt conflicts for:
schrodinger::pymol -> pyqt[version='5.*|>=5.6.0,<6.0a0|>=5.9.2,<6.0a0']
jupyter -> qtconsole -> pyqt[version='>=5.9.2,<5.10.0a0']

Package libzlib conflicts for:
schrodinger::pymol -> tk=[build=x11tk*] -> libzlib[version='>=1.2.11,<1.3.0a0']
python=3.9 -> tk[version='>=8.6.12,<8.7.0a0'] -> libzlib[version='>=1.2.11,<1.

In [18]:
from ipymol import viewer as pymol

In [26]:
!conda install -c bioconda pmx_biobb 

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - pmx_biobb -> python[version='>=3.7,<3.8.0a0']

Your python: python=3.9

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda will not
change your p

In [5]:
import Bio.PDB as pdb
from Bio.PDB.QCPSuperimposer import QCPSuperimposer

import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [1]:
import sys
stdout = sys.stdout

import numpy as np
import copy
import base64

# Отображение структур
import IPython.display
import ipywidgets
from IPython.display import display, display_svg, SVG, Image, HTML

# Open Drug Discovery Toolkit
import oddt
import oddt.docking
import oddt.interactions

# Органика
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

pmx работает с питоном меньше 3ки,у меня не получилось его даунгрейдить

In [2]:
pdb_ids = ["1lmp"]

In [6]:
pdb_list = pdb.PDBList()

In [7]:
path = pdb_list.retrieve_pdb_file(pdb_ids[0], file_format="pdb")

In [20]:
path

'/Users/mac-julia/Documents/HSE_3_sem/drug_design_hse/Smolkina/ipynb/lm/pdb1lmp.ent'

In [8]:
struct = pdb.PDBParser().get_structure("s", path)

/Users/mac-julia/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1422.
  warnings.warn(


In [21]:
struct

<Structure id=s>

In [9]:
chain = struct[0].child_list[0].child_list

In [17]:
chain

[<Residue LYS het=  resseq=1 icode= >,
 <Residue VAL het=  resseq=2 icode= >,
 <Residue TYR het=  resseq=3 icode= >,
 <Residue ASP het=  resseq=4 icode= >,
 <Residue ARG het=  resseq=5 icode= >,
 <Residue CYS het=  resseq=6 icode= >,
 <Residue GLU het=  resseq=7 icode= >,
 <Residue LEU het=  resseq=8 icode= >,
 <Residue ALA het=  resseq=9 icode= >,
 <Residue ARG het=  resseq=10 icode= >,
 <Residue ALA het=  resseq=11 icode= >,
 <Residue LEU het=  resseq=12 icode= >,
 <Residue LYS het=  resseq=13 icode= >,
 <Residue ALA het=  resseq=14 icode= >,
 <Residue SER het=  resseq=15 icode= >,
 <Residue GLY het=  resseq=16 icode= >,
 <Residue MET het=  resseq=17 icode= >,
 <Residue ASP het=  resseq=18 icode= >,
 <Residue GLY het=  resseq=19 icode= >,
 <Residue TYR het=  resseq=20 icode= >,
 <Residue ALA het=  resseq=21 icode= >,
 <Residue GLY het=  resseq=22 icode= >,
 <Residue ASN het=  resseq=23 icode= >,
 <Residue SER het=  resseq=24 icode= >,
 <Residue LEU het=  resseq=25 icode= >,
 <Residue

In [23]:
res_names = [residue.resname for residue in chain]
res_names

['LYS',
 'VAL',
 'TYR',
 'ASP',
 'ARG',
 'CYS',
 'GLU',
 'LEU',
 'ALA',
 'ARG',
 'ALA',
 'LEU',
 'LYS',
 'ALA',
 'SER',
 'GLY',
 'MET',
 'ASP',
 'GLY',
 'TYR',
 'ALA',
 'GLY',
 'ASN',
 'SER',
 'LEU',
 'PRO',
 'ASN',
 'TRP',
 'VAL',
 'CYS',
 'LEU',
 'SER',
 'LYS',
 'TRP',
 'GLU',
 'SER',
 'SER',
 'TYR',
 'ASN',
 'THR',
 'GLN',
 'ALA',
 'THR',
 'ASN',
 'ARG',
 'ASN',
 'THR',
 'ASP',
 'GLY',
 'SER',
 'THR',
 'ASP',
 'TYR',
 'GLY',
 'ILE',
 'PHE',
 'GLN',
 'ILE',
 'ASN',
 'SER',
 'ARG',
 'TYR',
 'TRP',
 'CYS',
 'ASP',
 'ASP',
 'GLY',
 'ARG',
 'THR',
 'PRO',
 'GLY',
 'ALA',
 'LYS',
 'ASN',
 'VAL',
 'CYS',
 'GLY',
 'ILE',
 'ARG',
 'CYS',
 'SER',
 'GLN',
 'LEU',
 'LEU',
 'THR',
 'ASP',
 'ASP',
 'LEU',
 'THR',
 'VAL',
 'ALA',
 'ILE',
 'ARG',
 'CYS',
 'ALA',
 'LYS',
 'ARG',
 'VAL',
 'VAL',
 'LEU',
 'ASP',
 'PRO',
 'ASN',
 'GLY',
 'ILE',
 'GLY',
 'ALA',
 'TRP',
 'VAL',
 'ALA',
 'TRP',
 'ARG',
 'LEU',
 'HIS',
 'CYS',
 'GLN',
 'ASN',
 'GLN',
 'ASP',
 'LEU',
 'ARG',
 'SER',
 'TYR',
 'VAL',
 'ALA',


In [4]:
def prepareImage(ind, width=500, height=500, sleep=2):
    cmd.ray(width, height)
    cmd.png('pymolimg%d.png' % ind)
    time.sleep(sleep)

def generate_html(title, ind):
    image_file = 'pymolimg%s.png' % ind
    html_code = '<h1>%s</h1><br><img src=\"data:image/jpeg;base64,%s\"/>'
    img_encoding = ''
    with open(image_file, 'rb') as f:
        img_encoding = f.read().encode('base64')
    
    return html_code % (title, img_encoding)

In [24]:
# Создание объектов белка и лиганда
newpdb = pdb.PDBList()
for r in res_names[-3:]:
    newpdb.remove_residue(r)
newpdb.writePDB('model.pdb')

lig = pdb.PDBList()
del lig.residues[:-3]
ligand_center = sum(map(lambda atom: np.array(atom.x), lig.atoms)) / len(lig.atoms)

AttributeError: module 'Bio.PDB' has no attribute 'copy'

In [ ]:
coordinates = np.zeros((len(lig.atoms),3))
for c,a in enumerate(lig.atoms):
    coordinates[c,:] = a.x
geom_center = np.mean(coordinates, axis = 0)
print geom_center

In [ ]:
newpdb.writePDB('new.pdb')
lig.writePDB('ligand.pdb')

In [27]:
prot = oddt.toolkit.readfile('pdb','model.pdb').next()

prot.OBMol.AddPolarHydrogens()
prot.OBMol.AutomaticPartialCharge()

OSError: No such file: 'model.pdb'